In [1]:
from pathlib import Path
import sys  
import os
import pandas as pd 
from datetime import datetime

import numpy as np
from operator import itemgetter

In [2]:
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..", "libs")))
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..", "src")))

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
from Utils.load_URM import load_URM
from Utils.load_ICM import load_ICM

from scipy.sparse import hstack, vstack
from Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

URM_all = load_URM("../data/data_train.csv")
ICM_all = load_ICM("../data/data_ICM_metadata.csv")

In [6]:
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.80)

In [7]:
evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])

EvaluatorHoldout: Ignoring 160 ( 0.4%) Users that have less than 1 test interactions


In [8]:
from libs.Recommenders.EASE_R.EASE_R_Recommender import EASE_R_Recommender

In [19]:
results = pd.DataFrame()

In [20]:
save_path = "result_experiments/EASE_R/"

In [22]:
# if os.path.exists(save_path+"results.csv"):
#     results = pd.read_csv(save_path+"results.csv")
# else:
#     os.makedirs(save_path)

In [23]:
URM_train_coo = URM_train.tocoo()

In [ ]:
max_MAP = -1

In [ ]:
for topK in [10, 30, 50, 70, 100, 200, 300, 400, None]:
    for l2_norm in [10, 30, 50, 100, 500, 1000, 2000, 3000, 4000]:
        for normalize_matrix in [True, False]:
            model_ease = EASE_R_Recommender(URM_train)

            current_timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            params = {
                "topK": topK,
                "l2_norm": l2_norm,
                "normalize_matrix": normalize_matrix,
                "time": current_timestamp
            }
            
            model_ease.fit(topK=topK, l2_norm=l2_norm, normalize_matrix=normalize_matrix)

            metrics_df, _ = evaluator_validation.evaluateRecommender(model_ease)
            if max_MAP < metrics_df["MAP"][10]:
                max_MAP = metrics_df["MAP"][10]
            print(', '.join(f"{key}: {value}" for key, value in params.items()), "MAP: ", metrics_df["MAP"][10], "max MAP: ", max_MAP)
                  
            params_df = pd.DataFrame([params])
            iteration_results = pd.concat([params_df, metrics_df.reset_index()], axis=1)
            results = pd.concat([results, iteration_results], ignore_index=True)
            results.to_csv(save_path+"results.csv", index=False)

In [9]:
ease = EASE_R_Recommender(URM_train)

In [10]:
ease.fit(topK=None, l2_norm = 45, normalize_matrix = False)

EASE_R_Recommender: Fitting model... 
EASE_R_Recommender: Fitting model... done in 46.68 min


In [12]:
ease.save_model("result_experiments/EASE_R/", 'ease_l245_weights')

EASE_R_Recommender: Saving model in file 'result_experiments/EASE_R/ease_l245_weights'
EASE_R_Recommender: Saving complete


In [13]:
result_dict, _ = evaluator_validation.evaluateRecommender(ease)

EvaluatorHoldout: Processed 35576 (100.0%) in 24.84 sec. Users per second: 1432


In [14]:
result_dict

,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.099398,0.149848,0.128295,0.052053,0.077257,0.283543,0.146298,0.112013,0.542332,0.378402,...,0.995523,0.539904,0.995523,0.157081,12.686545,0.999625,0.258119,0.859389,1.985036,0.234347
